In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
#telechargement des donnees de training et de testing
train_set=datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_set=datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
print(type(train_set))
print(type(test_set))

<class 'torchvision.datasets.mnist.FashionMNIST'>
<class 'torchvision.datasets.mnist.FashionMNIST'>


In [4]:
trainLoader=DataLoader(train_set, batch_size=64)
testLoader=DataLoader(test_set, batch_size=64)
for x, y in testLoader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [5]:
#creation du modele
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

cuda


In [7]:
class ReseauNeuronal(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear_relu= nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        X=self.flatten(x)
        logits=self.linear_relu(X)
        return logits

model = ReseauNeuronal().to(device)
print(model)

ReseauNeuronal(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss = nn.CrossEntropyLoss()
optimizer= torch.optim.SGD(model.parameters(), lr=1e-3)

#Fonction de trainning sur le model
def train(trainLoader, model, loss, optimizer):
    #Init les variables
    size=len(trainLoader.dataset)
    #train le modele
    model.train()
    #iter chacune des donnees sur le dataset pour entrainer le model
    for batch, (X, y) in enumerate(trainLoader):
        #on fait passer les tensor sur le device actuel
        X, y = X.to(device), y.to(device)
        #Entraine le modele
        pred= model(X)
        #Calcul des erreurs sur les predictions
        loss_train= loss(pred, y)
        
        #la Backpropagation
        #init l'optimizer
        optimizer.zero_grad()
        #la back pour le loss
        loss_train.backward()
        #on boucle l'optimization cette etape de l'opt
        optimizer.step()
        
        if batch%100 == 0: 
            loss_train, current = loss_train.item(), batch*len(X)
            print(f"loss: {loss_train:>7f} [{current:>5d}/{size:>5d}]")
            
            
#Fonction de test sur le model
def test(testLoader, model, loss):
    #initalisation des variables
    size=len(testLoader.dataset)
    num_batch=len(testLoader)
    #On evalue le model
    model.eval()
    #Variable pour le calcul des accuracy et le loss du modele
    testLoss, correct=0, 0
    with torch.no_grad():
        for X, y in testLoader:
            X, y= X.to(device), y.to(device)
            pred=model(X)
            testLoss+=loss(pred, y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    testLoss/=num_batch
    correct/=size
    print("Accuracy: ", correct, " error: ", testLoss)

In [9]:
for i in range(1, 5):
    print("New Epoch ",i, "----------------------")
    train(trainLoader, model, loss, optimizer)
    test(testLoader, model, loss)
    print('DONE !')

New Epoch  1 ----------------------
loss: 2.296194 [    0/60000]
loss: 2.288441 [ 6400/60000]
loss: 2.271704 [12800/60000]
loss: 2.269954 [19200/60000]
loss: 2.242206 [25600/60000]
loss: 2.230512 [32000/60000]
loss: 2.230683 [38400/60000]
loss: 2.196645 [44800/60000]
loss: 2.190757 [51200/60000]
loss: 2.169016 [57600/60000]
Accuracy:  0.5162  error:  2.156654643405015
DONE !
New Epoch  2 ----------------------
loss: 2.165758 [    0/60000]
loss: 2.161873 [ 6400/60000]
loss: 2.104283 [12800/60000]
loss: 2.119757 [19200/60000]
loss: 2.069596 [25600/60000]
loss: 2.019304 [32000/60000]
loss: 2.045591 [38400/60000]
loss: 1.967601 [44800/60000]
loss: 1.968678 [51200/60000]
loss: 1.906802 [57600/60000]
Accuracy:  0.5626  error:  1.899070382118225
DONE !
New Epoch  3 ----------------------
loss: 1.937344 [    0/60000]
loss: 1.912834 [ 6400/60000]
loss: 1.792481 [12800/60000]
loss: 1.826494 [19200/60000]
loss: 1.722446 [25600/60000]
loss: 1.679932 [32000/60000]
loss: 1.704105 [38400/60000]
loss: